<a href="https://colab.research.google.com/github/realalyeasin/deep_learning/blob/main/pytorch_resnet18_image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import os

In [40]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [41]:
data_dir = '/content/drive/MyDrive/Database/FlowerDataset/FlowerDataset'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'valid']}

In [42]:
data_loaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'valid']}

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [44]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

print(dataset_sizes)
print(class_names)

{'train': 1275, 'valid': 546}
['daisy', 'dandelion']


In [45]:
model = models.resnet18(pretrained= True)

In [46]:
for name, param in model.named_parameters():
  if 'fc' in name:
    param.requires_grad = True
  else:
    param.requires_grad = False

In [47]:
criteria = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=.001, momentum=.9)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [51]:
num_epochs = 10
for epoch in range(num_epochs):
  print(f'Epoch = {epoch}')
  for phase in ['train', 'valid']:
    if phase == 'train':
        model.train()
    else:
        model.eval()

    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in data_loaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criteria(outputs, labels)

            if phase == 'train':
                loss.backward()
                optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]

    print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')


  print('----------------------------------------------')
print("Finished!")

Epoch = 0
train Loss: 0.4673 Acc: 0.8282
valid Loss: 0.2972 Acc: 0.8901
----------------------------------------------
Epoch = 1
train Loss: 0.5568 Acc: 0.7929
valid Loss: 0.2041 Acc: 0.9139
----------------------------------------------
Epoch = 2
train Loss: 0.4373 Acc: 0.8322
valid Loss: 0.2030 Acc: 0.9176
----------------------------------------------
Epoch = 3
train Loss: 0.4298 Acc: 0.8188
valid Loss: 0.1744 Acc: 0.9231
----------------------------------------------
Epoch = 4
train Loss: 0.4021 Acc: 0.8376
valid Loss: 0.1894 Acc: 0.9194
----------------------------------------------
Epoch = 5
train Loss: 0.4746 Acc: 0.8055
valid Loss: 0.1614 Acc: 0.9359
----------------------------------------------
Epoch = 6
train Loss: 0.4536 Acc: 0.8133
valid Loss: 0.2816 Acc: 0.8883
----------------------------------------------
Epoch = 7
train Loss: 0.4691 Acc: 0.8306
valid Loss: 0.1501 Acc: 0.9341
----------------------------------------------
Epoch = 8
train Loss: 0.5096 Acc: 0.8118
valid L